# 01 — Data Acquisition & Cleaning

Here we focus on one essential goal: **building a clean daily price dataset** for the indices **IBEX 35 (Europe)** and **S&P 500 (United States)**.

The objective is to make sure it’s usable for modeling by aligning calendars, making consistent returns, and ensuring no gaps.

### Main steps
1. Retrieve daily data (2000 – today) from a reliable source.  
2. Align both indices on a common calendar.  
3. Handle missing observations and fill only where justified.  
4. Save clean CSVs for later stages.

### Outputs
- `/data/raw/IBEX.csv`  
- `/data/raw/SPX.csv`  
- `/data/interim/IBEX_clean.csv`  
- `/data/interim/SPX_clean.csv`


In [12]:
import yfinance as yf
import pandas as pd
from pathlib import Path

# Project directory structure relative paths only

PROJECT_ROOT = Path("..")
DATA_DIR     = PROJECT_ROOT / "data"
RAW          = DATA_DIR / "raw"
INTERIM      = DATA_DIR / "interim"

# Create folders if missing
RAW.mkdir(parents=True, exist_ok=True)
INTERIM.mkdir(parents=True, exist_ok=True)

print("RAW directory     →", RAW)
print("INTERIM directory →", INTERIM)


RAW directory     → ../data/raw
INTERIM directory → ../data/interim


In [13]:
# Download daily close prices for IBEX 35 and S&P 500

symbols = {
    "IBEX": "^IBEX",
    "SPX": "^GSPC"
}

start_date = "2000-01-01"

for name, ticker in symbols.items():
    df = yf.download(ticker, start=start_date, progress=False)

    # Reset index so Date becomes a column
    df = df.reset_index()[["Date", "Close"]]
    df = df.rename(columns={"Close": name})

    # Save raw file
    df.to_csv(RAW / f"{name}.csv", index=False)
    print(f"{name} downloaded → {len(df)} rows → {RAW / f'{name}.csv'}")


/tmp/ipykernel_19474/1603910352.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, progress=False)


IBEX downloaded → 6593 rows → ../data/raw/IBEX.csv


/tmp/ipykernel_19474/1603910352.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, progress=False)


SPX downloaded → 6518 rows → ../data/raw/SPX.csv


### 3. Aligning and Cleaning Both Indices

The IBEX 35 and S&P 500 don’t share identical trading calendars, so to make them directly comparable, both series must be aligned to a common business-day index.  

This step:
- Reads the raw CSVs and removes metadata rows.  
- Converts date columns into a consistent `datetime` format.  
- Reindexes both datasets to a shared daily business calendar.  
- Forward-fills only short gaps in trading days (for instance, holidays that differ between Europe and the U.S.).  
- Exports clean datasets to `/data/interim/` for later use.  


In [15]:

# Step 3 — Align, Clean, and Build Unified Dataset


ibex = pd.read_csv(RAW / "IBEX.csv", parse_dates=["Date"])
spx  = pd.read_csv(RAW / "SPX.csv",  parse_dates=["Date"])

merged = (
    ibex.merge(spx, on="Date", how="inner")
        .sort_values("Date")
        .drop_duplicates(subset="Date")
        .reset_index(drop=True)
)

# Keep only numeric columns + Date
numeric_cols = merged.select_dtypes(include="number").columns
clean = merged[["Date"] + list(numeric_cols)]

print("Columns used:", clean.columns.tolist())

# Save aligned datasets
clean.to_csv(INTERIM / "merged_clean.csv", index=False)


# Compute returns

returns = (
    clean.set_index("Date")
         .pct_change()
         .dropna()
)

returns.to_csv(INTERIM / "returns.csv")

print("Returns shape:", returns.shape)


Columns used: ['Date']
Returns shape: (6423, 0)
